<a href="https://colab.research.google.com/github/wilson1yan/VideoGPT/blob/master/notebooks/Using_VideoGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using VideoGPT
This is a notebook demonstrating how to use VideoGPT and any pretrained models, Make sure that it is a GPU instance: **Change Runtime Type -> GPU**

## Installation
First, we install the necessary packages

In [ ]:
! pip install git+https://github.com/wilson1yan/VideoGPT.git
! pip install scikit-video av

In [2]:
%matplotlib inline

from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML

import os
import torch
from torchvision.io import read_video, read_video_timestamps

from videogpt import download, load_vqvae, load_videogpt
from videogpt.data import preprocess

VIDEOS = {
    'breakdancing': '1OZBnG235-J9LgB_qHv-waHZ4tjofiDgj',
    'bear': '16nIaqq2vbPh-WMo_7hs9feVSe0jWVXLF',
    'jaywalking': '1UxKCVrbyXhvMz_H7dI4w5hjPpRGCAApy',
    'cartoon': '1ONcTMSEuGuLYIDbX-KeFqd390vbTIH9d'
}

## Downloading a Pretrained VQ-VAE
There are four pretrained models available: `bair_stride4x2x2`, `ucf101_stride4x4x4`, `kinetics_stride4x4x4`, and `kinetics_stride2x4x4`. BAIR was trained on 64 x 64 video, and the rest on 128 x 128. The `stride` component represents the THW downsampling the VQ-VAE performs on the video tensor.

In [3]:
device = torch.device('cuda')
vqvae = load_vqvae('kinetics_stride2x4x4', device=device).to(device)

258MiB [00:12, 21.5MiB/s]


## Video Loading and Preprocessing
The code below downloads, loads, and preprocesses a given `mp4` file.

In [4]:
video_name = 'jaywalking'
# `resolution` must be divisible by the encoder image stride
# `sequence_length` must be divisible by the encoder temporal stride
resolution, sequence_length = vqvae.args.resolution, 16

video_filename = download(VIDEOS[video_name], f'{video_name}.mp4')
pts = read_video_timestamps(video_filename, pts_unit='sec')[0]
video = read_video(video_filename, pts_unit='sec', start_pts=pts[0], end_pts=pts[sequence_length - 1])[0]
video = preprocess(video, resolution, sequence_length).unsqueeze(0).to(device)

3.29MiB [00:00, 108MiB/s]


## VQ-VAE Encoding and Decoding
Now, we can encode the video through the `encode` function. The `encode` function also has an optional input `including_embeddings` (default `False`) which will also return the embedding versions of the encodings.

In [5]:
with torch.no_grad():
    encodings = vqvae.encode(video)
    video_recon = vqvae.decode(encodings)
    video_recon = torch.clamp(video_recon, -0.5, 0.5)

## Visualizing Reconstructions

In [6]:
videos = torch.cat((video, video_recon), dim=-1)
videos = videos[0].permute(1, 2, 3, 0) # CTHW -> THWC
videos = ((videos + 0.5) * 255).cpu().numpy().astype('uint8')

fig = plt.figure()
plt.title('real (left), reconstruction (right)')
plt.axis('off')
im = plt.imshow(videos[0, :, :, :])
plt.close()

def init():
    im.set_data(videos[0, :, :, :])

def animate(i):
    im.set_data(videos[i, :, :, :])
    return im

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=videos.shape[0], interval=200) # 200ms = 5 fps
HTML(anim.to_html5_video())

# Using Pretrained VideoGPT Models

The current available model to download is `ucf101`.

In [7]:
device = torch.device('cuda')
gpt = load_videogpt('ucf101_uncond_gpt', device=device).to(device)

1.81GiB [00:21, 84.8MiB/s]
258MiB [00:06, 43.0MiB/s]
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:552: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")


`VideoGPT.sample` method return generated sample of shape BCTHW in the range [0, 1]

In [8]:
samples = gpt.sample(16) # unconditional model does not require batch input

100%|██████████| 4096/4096 [07:41<00:00,  8.87it/s]


In [9]:
import math
import numpy as np

b, c, t, h, w = samples.shape
samples = samples.permute(0, 2, 3, 4, 1)
samples = (samples.cpu().numpy() * 255).astype('uint8')

video = np.zeros((t, (1 + h) * 4 + 1, (1 + w) * 4 + 1, c), dtype='uint8')
for i in range(b):
  r, c = i // 4, i % 4
  start_r, start_c = (1 + h) * r, (1 + w) * c
  video[:, start_r:start_r + h, start_c:start_c + w] = samples[i]

fig = plt.figure()
plt.title('ucf101 unconditional samples')
plt.axis('off')
im = plt.imshow(video[0, :, :, :])
plt.close()

def init():
    im.set_data(video[0, :, :, :])

def animate(i):
    im.set_data(video[i, :, :, :])
    return im

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=video.shape[0], interval=200) # 200ms = 5 fps
HTML(anim.to_html5_video())